In [4]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    get_load_config_from_yaml,
)
from importlib import reload
import core
import keras
import core.keras_models.RegressionTransformer as RegressionTransformer

PLOTS_DIR = f"plots/regression_transformer/"
MODEL_DIR = f"models/regression_transformer/"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


data_config = DataProcessor.load_from_npz(
    load_config.data_path["nominal"], max_events=4_000_000, event_numbers="even"
)

X_train, y_train = DataProcessor.get_data()
del DataProcessor  # Free memory

2026-02-03 13:30:21.923647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770121821.946078 1980612 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770121821.953762 1980612 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770121821.971396 1980612 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770121821.971414 1980612 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770121821.971417 1980612 computation_placer.cc:177] computation placer alr

In [5]:
reload(RegressionTransformer)
Transformer = RegressionTransformer.FeatureConcatTransformerReconstructor(data_config, name="Transformer")

In [6]:
Transformer.build_model(
    hidden_dim=64,
    num_layers=6,
    dropout_rate=0.1,
)


I0000 00:00:1770121955.030102 1980612 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10477 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


In [ ]:
Transformer.adapt_normalization_layers(X_train)
Transformer.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(),
        "normalized_regression": core.utils.MagnitudeDirectionLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4, clipnorm=1.0),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy(name="assignment_accuracy")],
        "normalized_regression": [core.utils.RegressionDeviation()],
    },
    #add_physics_informed_loss=True,
    loss_weights={"assignment": 1.0, "normalized_regression": 1.0}
)

I0000 00:00:1770121960.865866 1983109 service.cc:152] XLA service 0x7f5d50002ec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770121960.865892 1983109 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2026-02-03 13:32:40.891140: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1770121960.947038 1983109 cuda_dnn.cc:529] Loaded cuDNN version 91500
I0000 00:00:1770121961.231808 1983109 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Adapted normalization layer:  hlf_input_normalization
Adapted normalization layer:  met_input_normalization
Adapted normalization layer:  lep_input_normalization
Adapted normalization layer:  jet_input_normalization
Set regression denormalization layer with computed mean and std.


Model: "keras_model_wrapper_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ jet_inputs          │ (None, 6, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_transform │ (None, 6, 5)      │          0 │ jet_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6)         │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_transform │ (None, 2, 4)      │          0 │ lep_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_inputs          │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ compute_high_level… │ (None, 6, 2, 2)   │          0 │ jet_input_transf… │
│ (ComputeHighLevelF… │                   │            │ jet_mask[0][0],   │
│                     │                   │            │ lep_input_transf… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_transform │ (None, 1, 2)      │          0 │ met_inputs[0][0]  │
│ (InputMetPhiLayer)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hlf_input_normaliz… │ (None, 6, 2, 2)   │          9 │ compute_high_lev… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_normaliz… │ (None, 1, 2)      │          5 │ met_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_normaliz… │ (None, 2, 4)      │          9 │ lep_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_normaliz… │ (None, 6, 5)      │         11 │ jet_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 6, 4)      │          0 │ hlf_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 2)         │          0 │ met_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 8)         │          0 │ lep_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 9)      │          0 │ jet_input_normal… │
│ (Concatenate)       │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 6, 2)      │          0 │ flatten[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 462,759 (1.77 MB)

 Trainable params: 462,725 (1.77 MB)

 Non-trainable params: 34 (152.00 B)

In [ ]:
Transformer.train_model(
    epochs=10,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
    ],
    validation_split=0.1,
)

Epoch 1/10
3511/3512 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - assignment_assignment_accuracy: 0.1843 - assignment_loss: 0.1766 - loss: 3.2481 - normalized_regression_deviation: 0.7187 - normalized_regression_loss: 3.0715

2026-02-03 13:36:38.166230: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_11', 124 bytes spill stores, 104 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_12', 112 bytes spill stores, 92 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_13', 112 bytes spill stores, 92 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_1', 16 bytes spill stores, 16 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_8', 16 bytes spill stores, 16 bytes spill loads



3512/3512 ━━━━━━━━━━━━━━━━━━━━ 226s 44ms/step - assignment_assignment_accuracy: 0.1844 - assignment_loss: 0.1766 - loss: 3.2480 - normalized_regression_deviation: 0.7187 - normalized_regression_loss: 3.0714 - val_assignment_assignment_accuracy: 0.2819 - val_assignment_loss: 0.1615 - val_loss: 2.8118 - val_normalized_regression_deviation: 0.6707 - val_normalized_regression_loss: 2.6503 - learning_rate: 1.0000e-04
Epoch 2/10
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 115s 33ms/step - assignment_assignment_accuracy: 0.2365 - assignment_loss: 0.1702 - loss: 2.7236 - normalized_regression_deviation: 0.6426 - normalized_regression_loss: 2.5534 - val_assignment_assignment_accuracy: 0.2633 - val_assignment_loss: 0.1621 - val_loss: 2.5442 - val_normalized_regression_deviation: 0.6240 - val_normalized_regression_loss: 2.3819 - learning_rate: 1.0000e-04
Epoch 3/10
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 116s 33ms/step - assignment_assignment_accuracy: 0.2421 - assignment_loss: 0.1673 - loss: 2.5625 - normalized_regre

In [ ]:
Transformer.save_model(MODEL_DIR + "odd_model.keras")

In [ ]:
upscale_layer = Transformer.model.get_layer("regression")

In [ ]:
upscale_layer(y_train["normalized_regression"])

In [ ]:
import numpy as np
import tensorflow as tf
np.abs(upscale_layer(y_train["normalized_regression"]) - X_train["regression"]).mean()

In [ ]:
from core.components import reco_W_mass_deviation
leptons = X_train["lep_inputs"]
neutrino_pred = Transformer.reconstruct_neutrinos(X_train)
neutrino_true = X_train["regression"]
neutrino_nu_flows = X_train["nu_flows_neutrino_truth"]

In [ ]:
reco_W_mass_deviation(tf.convert_to_tensor(neutrino_pred, dtype=tf.float32), tf.convert_to_tensor(leptons, dtype=tf.float32)).numpy().mean()

In [ ]:
reco_W_mass_deviation(tf.convert_to_tensor(neutrino_true, dtype=tf.float32), tf.convert_to_tensor(leptons, dtype=tf.float32)).numpy().mean()

In [ ]:
reco_W_mass_deviation(tf.convert_to_tensor(neutrino_nu_flows, dtype=tf.float32), tf.convert_to_tensor(leptons, dtype=tf.float32)).numpy().mean()